# A simple test for inquiry model


Here is the setting:

- My thought: Yintai Ma [09:23] 
  - Suppose I have a video X of 100 frames, denoting as X[0] to X[99]. We then split it to multiple clips without overlap. If then length of each clip is 5 frames, then we come up with a sequences of clips X[0:5],X[5:10],...,X[95:100]. We use the encoder to transfer the sequences of clips to sequences of embeddeds, reads y[0],y[1],...,y[20]. Now we randomly pick some frames from X as validation clip and transfer it into validation embedded y_hat. We want to see how y_hat is matching to the sequences of y[0]...y[20].

- Diego:
  - You have video X. Randomly pick  sequences of frames (non overlapping). Say x[4:10], x[34:37],x[85:95]. Now concatenate them into a single video. This is now your query. From here, create embedding sequence y[0],…y[K]. Now do DTW of y against your encoded sequences of the videos in the database.
  - This is subject to experimentation. I agree that there should be overlap. Overlap by half.

- My thought: There are many variations for the implementation:
  1. we keep overlaps for y. We transfer X[0:5],X[1:6],....X[94:99],X[95:100] into y[0]...y[100]. Now we want to see how y_hat is matching to y[].
  2. When we pick some frames from X to composite validation clips, do we always pick 5 consecutive frames? Should we ever transfer X[0]+X[2:6] into y_hat?
  3. How we match y_hat to y[] if y_hat is also a sequences? I think this is where DTW comes in right? If y_hat is just one embedded, then what we need is basically a simple comparison between the distance of two embedded. However, if y_hat is a sequences, say it has y_hat[0] and y_hat[1], then we will need to use DTW to consider the case where both y_hat[0] matches to y[0] and y_hat[1] matches to y[3] are the best query retrieve.


In [1]:
from keras.models import Sequential, Model
from keras.utils import plot_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, LSTM, RepeatVector
from keras.layers.wrappers import *
from keras.layers.core import *
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.optimizers import RMSprop
from keras import backend as K

import csv
import os
import time
import sys
import numpy as np
import matplotlib
# matplotlib.use('Agg')
import pylab as plt
from scipy.misc import toimage

import imageio
import cv2
import numpy as np
import numpngw
import pandas as pd 

from IPython.display import HTML
import random

Using TensorFlow backend.
/home/lab.analytics.northwestern.edu/yma/.conda/envs/py3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Test Data Generation Function

In [2]:
GENERATE_DATA = 1 
LOG_DIR = "../../tensorboard/log/"
EPOCH = 150
sequenceLength = 3
setup_name = "clrmvsq_simple_vgg_a"
N_SAMPLES = 1000
BATCHSIZE = 5
ucf_generate_fps = 2  # The fps to sample from the original UCF data to generate the train and val set
data_path = "../../data/UCF/"

batch_size=20
data_type = 'images'
concat=False


In [3]:
import data_seq
data_seq = reload(data_seq)

data = data_seq.DataSet(seq_length=5,class_limit=10)

print data.data[:1], '\n'
print data.classes[:5]
print data.image_shape


SyntaxError: Missing parentheses in call to 'print' (<ipython-input-3-55990598bef0>, line 6)

In [ ]:
generator = data.seq_generator(batch_size, 'train', 'images')

X,y  = next(generator);

print X.shape

images = X[18]
imageio.mimsave('./movie.gif', images)
HTML('<img src="./movie.gif">')

In [ ]:
X,y  = next(generator);
for i in range (0,5):
    images = X[i][0]
    plt.imshow(images)
    plt.show()

In [ ]:
for _ in range(10):
    X,y  = next(generator);

In [ ]:

X,y  = next(generator);

print X.shape

images = X[18]
imageio.mimsave('./movie.gif', images)
HTML('<img src="./movie.gif">')

# Adjust the saved model

In [ ]:
import sys,os,os.path
sys.path.append(os.path.expanduser('/home/lab.analytics.northwestern.edu/yma/git/videodl/seq_inquiry'))
os.environ['CUDA_VISIBLE_DEVICES']='2'

In [ ]:
# Examaine model file
from keras.models import load_model

# weights_file = "/home/lab.analytics.northwestern.edu/yma/git/data/checkpoints/ucf_vgg16_seq3_convlstm.040-0.0857.hdf5"

weights_file = "/home/lab.analytics.northwestern.edu/yma/git/data/checkpoints/ucf_vgg16_simple_seq3_convlstm.001-0.03.hdf5"

model = load_model(weights_file)

In [ ]:
json_string = model.to_json()

# Test Model Validation

In [ ]:
print("--- Start Assembling the Model ---")

inputs = Input(shape=(sequenceLength,224,224,3))

# conved = TimeDistributed(Lambda(MyCNN), input_shape=(sequenceLength,40,40,1)) (inputs)

x = TimeDistributed(Conv2D(64, (3, 3), padding='same', activation='relu', name='block1_conv1'), input_shape=(sequenceLength,224,224,3))(inputs)
x = TimeDistributed(MaxPooling2D((2, 2), name='block1_pool'))(x)
# x = TimeDistributed(MaxPooling2D((2, 2)))(x)
x = TimeDistributed(Conv2D(128, (3, 3), padding='same', activation='relu', name='block2_conv1'))(x)
x = TimeDistributed(MaxPooling2D((2, 2), name='block2_pool'))(x)
x = TimeDistributed(Conv2D(256, (3, 3), padding='same', activation='relu', name='block3_conv1'))(x)
x = TimeDistributed(MaxPooling2D((2, 2), name='block3_pool'))(x)
x = TimeDistributed(Conv2D(512, (3, 3), padding='same', activation='relu', name='block4_conv1'))(x)
x = TimeDistributed(MaxPooling2D((2, 2), name='block4_pool'))(x)
x = TimeDistributed(Conv2D(512, (3, 3), padding='same', activation='relu', name='block5_conv1'))(x)

# LSTM part
x = TimeDistributed(MaxPooling2D((2, 2), name='block5_pool'))(x)
# x = TimeDistributed(Flatten())(x)

encoded = ConvLSTM2D(filters=100, kernel_size=(3, 3),padding='same', return_sequences=True)(x)

encoder = Model(output=encoded,input=inputs)
# myoptmizer = RMSprop(lr=0.1, decay=1e-4)
# autoencoder.compile(loss='mean_squared_error', optimizer=myoptmizer)
encoder.compile(loss='mean_squared_error', optimizer='RMSprop')

plot_model(encoder, to_file='query_model.png', show_shapes=True)

print('--- Finish Compile and Plot Model ---')

In [ ]:
weights_file = "/home/lab.analytics.northwestern.edu/yma/git/data/checkpoints/ucf_simple_vgg16_seq3_convlstm_c5.002-0.0186.hdf5"
encoder.load_weights(weights_file, by_name=True)

## Code Start from here

In [ ]:

def create_database(data):
    database = []
    for i in range(0, N_database):
        smp = data.data[i]
        vid_len = int(smp[3])
        frams = data.get_frames_for_sample(smp)
        frams = data.rescale_list(frams, vid_len-1)
        seq = data.build_image_sequence(frams)
        seq = np.array(seq)
        
        seqX = []
        for j in range(0,len(seq)-SEQ_LENGTH, (SEQ_LENGTH>>1)+1):
            seqX.append(seq[j:j+SEQ_LENGTH])
        seqX = np.array(seqX)
        seqY = encoder.predict(seqX)
        database.append((seqY, smp[2]))
    return database
        

In [ ]:
import random
def get_random_inquiry(data, if_random=True):
    if if_random:
        smp = random.choice(data.data)
    else:
        smp = data.data[0]
    
    vid_len = int(smp[3])
    frams = data.get_frames_for_sample(smp)
    frams = data.rescale_list(frams, vid_len-1)
    seq = data.build_image_sequence(frams)
    seq = np.array(seq)
    
    seqX = []
    X_frames_start= []
    stepsize = vid_len / inq_length - 2
    for j in range(0, inq_length):
        jst = j*stepsize
        jend = j*stepsize+SEQ_LENGTH
        seqX.append(seq[jst:jend])
        X_frames_start.append((jst, jend))
    seqX = np.array(seqX)
    seqY = encoder.predict(seqX)
    return (seqX, seqY ,smp[2], X_frames_start)


In [ ]:
def inquiry_in_database(inquiry_seqY, database):
    scores= []
    score_names = []
    bestscore = 1<<30
#     dist, cost, acc, path
    for i in database:
#         print(inquiry_seqY.shape)
#         print(i[0].shape)
        seqyflat = inquiry_seqY.reshape((inquiry_seqY.shape[0], 3*7*7*100))
        iyflat = i[0].reshape((i[0].shape[0], 3*7*7*100))
#         print(seqyflat.shape)
#         print(iyflat.shape)
        dist, cost, acc, path = dtw(seqyflat, iyflat, dist=lambda x, y: np.linalg.norm(x - y, ord=1))
        scores.append((dist,i[1]))
        if dist< bestscore:
            bestscore = dist
            bestpath = path
            bestacc= acc
            bestfilename = i[1]
    scores = sorted(scores, key=lambda x:x[0])
    return (bestscore, bestacc, bestpath, bestfilename, scores)

In [ ]:
import data_seq
from scipy.spatial.distance import euclidean
from dtw import dtw
%pylab inline

data_seq = reload(data_seq)
SEQ_LENGTH = 3
# the number of frames in each clip

data = data_seq.DataSet(seq_length=SEQ_LENGTH,class_limit=20, random_class=True)
random.shuffle(data.data)
N_database = 100

inq_length = 8
# the number of clips in the inquiry, no overlap

In [ ]:
database = create_database(data)

In [ ]:
inqs = get_random_inquiry(data, True)
seqY = inqs[1]

inq_result = inquiry_in_database(seqY, database)

In [ ]:
print("Inqury is:\t"+ inqs[2])
print("Inqury frames:\t"+ str(inqs[3]))
print("Best Match is:\t"+ inq_result[3])
print("Best Dist:\t"+ str(inq_result[0]))

acc = inq_result[1]
path = inq_result[2]
imshow(acc.T, origin='lower', cmap=cm.gray, interpolation='nearest')
plot(path[0], path[1], 'w')
# xlim((-0.5, acc.shape[0]-0.5))
# ylim((-0.5, acc.shape[1]-0.5))

print("========================")
print("All scores in database:")
for i in range(len(inq_result[4])):
    print("Record: "+ inq_result[4][i][1].ljust(25)+ "\tDTW Dist: "+ str(inq_result[4][i][0]))